# DS321: Computational Statistics <br>

##   Lecture: Metropolis-Hastings Algorithm

University of Science and Technology of Southern Philippines <br>

## Student Name: 

Instructor: **Romen Samuel Wabina, MSc** <br>
MSc Data Science and AI | Asian Institute of Technology <br>
*ongoing* PhD Data Science (Healthcare and Clinical Informatics) 

In [1]:
%matplotlib inline

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
import numpy as np

# Transition probabilities representing the Markov Chain
transition_prob = {
    'Stage 3': {'Stage 3': 0.5, 'Stage 4': 0.4, 'Stage 5': 0.1, 'CVD': 0.0, 'DEATH': 0.0},
    'Stage 4': {'Stage 3': 0.0, 'Stage 4': 0.6, 'Stage 5': 0.3, 'CVD': 0.1, 'DEATH': 0.0},
    'Stage 5': {'Stage 3': 0.0, 'Stage 4': 0.0, 'Stage 5': 0.7, 'CVD': 0.2, 'DEATH': 0.1},
    'CVD':     {'Stage 3': 0.0, 'Stage 4': 0.0, 'Stage 5': 0.0, 'CVD': 0.8, 'DEATH': 0.2},
    'DEATH':   {'Stage 3': 0.0, 'Stage 4': 0.0, 'Stage 5': 0.0, 'CVD': 0.0, 'DEATH': 1.0}
}

num_sequences = 1          # Number of sequences to generate
initial_state = 'Stage 3'  # Starting from Stage 3

# Loop over each sequence
for sequence_num in range(num_sequences):
    sequence = [initial_state]  # Store the states of the generated sequence
    current_state = initial_state  # Set the current state to the initial state

    # Generate the sequence until 'DEATH' state is reached
    while current_state != 'DEATH':
        # Randomly select the next state based on transition probabilities
        next_state = np.random.choice(list(transition_prob[current_state].keys()), p=list(transition_prob[current_state].values()))
        sequence.append(next_state)  # Append the next state to the sequence
        current_state = next_state   # Update the current state

    # Print the sequence label
    print("Scenario", sequence_num + 1, ":")

    # Iterate over each step in the sequence
    for i, state in enumerate(sequence):
        if i < len(sequence) - 1:  # Check if it's not the last step
            next_state = sequence[i + 1]  # Get the next state
            transition_prob_value = transition_prob[state][next_state]  # Get the transition probability
            # Print the step, current state, next state, and transition probability
            print("Step", i, ":\t", state, "->", next_state, "(Transition Probability:", transition_prob_value, ")")
        else:  # Last step in the sequence
            # Print the step and current state
            print("Step", i, ":\t", state)
    print()


Scenario 1 :
Step 0 :	 Stage 3 -> Stage 3 (Transition Probability: 0.5 )
Step 1 :	 Stage 3 -> Stage 4 (Transition Probability: 0.4 )
Step 2 :	 Stage 4 -> Stage 5 (Transition Probability: 0.3 )
Step 3 :	 Stage 5 -> Stage 5 (Transition Probability: 0.7 )
Step 4 :	 Stage 5 -> Stage 5 (Transition Probability: 0.7 )
Step 5 :	 Stage 5 -> Stage 5 (Transition Probability: 0.7 )
Step 6 :	 Stage 5 -> Stage 5 (Transition Probability: 0.7 )
Step 7 :	 Stage 5 -> Stage 5 (Transition Probability: 0.7 )
Step 8 :	 Stage 5 -> DEATH (Transition Probability: 0.1 )
Step 9 :	 DEATH



## Direct Sampling

Direct sampling, also known as direct Monte Carlo sampling or direct simulation, is a technique used in statistics and Monte Carlo methods to generate samples from a target distribution of interest. The goal of direct sampling is to obtain representative samples from a complex or high-dimensional distribution that may be challenging to sample from directly. In direct sampling, each sample is generated independently by drawing random values from the target distribution. The key idea is to use the properties of the target distribution, such as its probability density function (PDF), to guide the sampling process.

The general steps involved in direct sampling are as follows:
1. Define the target distribution: Specify the probability distribution from which you want to generate samples. This distribution may represent a population, a physical system, or any other phenomenon of interest.
2. Determine the sampling method: Choose a suitable technique to sample from the target distribution. This choice depends on the specific characteristics of the distribution and the available information about it. Some common direct sampling methods include inverse transform sampling, acceptance-rejection sampling, and importance sampling.
3. Generate independent samples: Using the selected sampling method, generate a series of independent samples from the target distribution. The number of samples generated depends on the desired sample size or the convergence criteria of the estimation problem at hand.
4. Analyze the samples: Once the samples are generated, they can be used for various purposes, such as estimating population statistics, simulating system behavior, or conducting statistical inference.


In MC estimation, we approximate an integral by the sample mean of a function of simulated random variables. In more mathematical terms,

$$\int p(x)\ f(x)\ dx = \mathbb{E}_{p(x)} \big[\ f(x) \big] \approx \frac{1}{N} \sum_{n=1}^{N}f(x_n)$$

where $x_n \sim \ p(x)$.

A useful application of MC is probability estimation. In fact, we can cast a probability as an expectation using the indicator function. In our case, given that $A = \{I \ | \ I > 275\}$, we define $f(x)$ as

$$f(x) = I_{A}(x)= \begin{cases} 
      1 & I \geq 275 \\
      0 & I < 275 
   \end{cases}$$
   
Replacing in our equation above, we get

$$\int p(x) \ f(x) \ dx = \int I(x)\ p(x) \ d(x) = \int_{x \in A} p(x)\ d(x) \approx \frac{1}{N} \sum_{n=1}^{N}I_{A}(x_n)$$

In [3]:
import numpy as np

def monte_carlo_markov_chain(transition_matrix, initial_state, num_samples, num_steps):
    samples = []
    
    for _ in range(num_samples):
        current_state = initial_state
        states = [current_state]

        for _ in range(num_steps):
            transition_probabilities = transition_matrix[current_state]

            # Direct Sampling
            next_state = np.random.choice(len(transition_probabilities), p = transition_probabilities)
            current_state = next_state
            states.append(current_state)

            # Stop sampling if a terminal state (e.g., Death) is reached
            if current_state == len(transition_matrix) - 1:
                break

        samples.append(states)
    return samples

# Transition matrix representing the Markov Chain
transition_matrix = np.array([
    [0.5, 0.4, 0.1, 0.0, 0.0],
    [0.0, 0.6, 0.3, 0.1, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.8, 0.2],
    [0.0, 0.0, 0.0, 0.0, 1.0]])

initial_state = 0     # Starting from Stage 3
num_samples   = 1000    # Number of Monte Carlo samples
num_steps     = 10     # Number of steps to sample

# Perform Monte Carlo sampling with the Markov Chain
sampled_states = monte_carlo_markov_chain(transition_matrix, initial_state, num_samples, num_steps)

# Print the sampled states for the first sample
print("Sampled States (First Sample):", sampled_states[0])

Sampled States (First Sample): [0, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3]


In [4]:
for idx, state in enumerate(sampled_states):
    print(f'Patient {idx} with CKD Progression: \t {set(state)}')

Patient 0 with CKD Progression: 	 {0, 1, 2, 3}
Patient 1 with CKD Progression: 	 {0, 1, 2, 4}
Patient 2 with CKD Progression: 	 {0, 1, 2, 4}
Patient 3 with CKD Progression: 	 {0, 1, 2, 3, 4}
Patient 4 with CKD Progression: 	 {0, 1, 2, 3, 4}
Patient 5 with CKD Progression: 	 {0, 2, 3, 4}
Patient 6 with CKD Progression: 	 {0, 1, 3, 4}
Patient 7 with CKD Progression: 	 {0, 1, 2}
Patient 8 with CKD Progression: 	 {0, 2, 3, 4}
Patient 9 with CKD Progression: 	 {0, 1, 2, 4}
Patient 10 with CKD Progression: 	 {0, 2, 3}
Patient 11 with CKD Progression: 	 {0, 1, 3, 4}
Patient 12 with CKD Progression: 	 {0, 1, 3, 4}
Patient 13 with CKD Progression: 	 {0, 1, 2, 3}
Patient 14 with CKD Progression: 	 {0, 1, 3, 4}
Patient 15 with CKD Progression: 	 {0, 1, 2}
Patient 16 with CKD Progression: 	 {0, 2, 4}
Patient 17 with CKD Progression: 	 {0, 2, 3}
Patient 18 with CKD Progression: 	 {0, 1, 3}
Patient 19 with CKD Progression: 	 {0, 1, 3, 4}
Patient 20 with CKD Progression: 	 {0, 1, 2, 4}
Patient 21 wit

In the context of a Markov Chain model for chronic kidney disease progression, direct sampling is not inherently difficult because the Markov Chain allows for the direct generation of samples. The transition probabilities between different stages of the disease and other health states determine the probabilities of transitioning from one state to another. However, direct sampling have disadvantages:

1. Direct sampling can become challenging in high-dimensional spaces due to the "curse of dimensionality." Obtaining representative samples becomes computationally expensive or infeasible as the number of dimensions increases.
2. In some cases, the distribution of CKD stages and associated risk factors may be analytically or computationally intractable, making direct sampling challenging. If the distributions cannot be easily parameterized or sampled from, alternative methods like importance sampling or MCMC techniques may be more suitable.
3. Direct sampling can be computationally expensive, especially if the CKD model involves complex calculations, simulations, or computationally intensive algorithms. Sampling directly from a large dataset or running extensive simulations may not be feasible in real-time applications or when dealing with a massive amount of data.

Direct sampling is particularly useful when the target distribution can be easily evaluated or sampled from directly. It is straightforward to implement and does not require additional steps or adjustments compared to more complex sampling techniques like Markov Chain Monte Carlo (MCMC). However, direct sampling may become impractical or inefficient when the target distribution is high-dimensional, has complex dependencies, or lacks an analytically tractable form. In such cases, advanced sampling methods like MCMC algorithms, including the Metropolis-Hastings algorithm or Gibbs sampling, are often employed to overcome these challenges.

## Importance Sampling

Importance sampling is a Markov Chain Monte Carlo method for evaluating properties of a particular distribution, while only having samples generated from a different distribution than the distribution of interest. 

Importance sampling is a variance reduction technique that can be used in the Monte Carlo method. The idea behind importance sampling is that certain values of the input random variables in a simulation have more impact on the parameter being estimated than others. 
- If these "important" values are emphasized by sampling more frequently, then the estimator variance can be reduced. Hence, the basic methodology in importance sampling is to choose a distribution which "encourages" the important values.


With importance sampling, we try to reduce the variance of our Monte-Carlo integral estimation by choosing a better distribution from which to simulate our random variables. It involves multiplying the integrand by 1 (usually dressed up in a “tricky fashion”) to yield an expectation of a quantity that varies less than the original integrand over the region of integration. Concretely,

$$\mathbb{E}_{p(x)} \big[\ f(x) \big] = \int f(x)\ p(x)\ dx = \int f(x)\ p(x)\ \frac{q(x)}{q(x)}\ dx = \int \frac{p(x)}{q(x)}\cdot f(x)\ q(x)\ dx = \mathbb{E}_{q(x)}  \big[\ f(x)\cdot \frac{p(x)}{q(x)} \big]$$

Thus, the MC estimation of the expectation becomes:

$$\mathbb{E}_{q(x)}  \big[\ f(x)\cdot \frac{p(x)}{q(x)} \big] \approx \frac{1}{N} \sum_{n=1}^{N} w_n \cdot f(x_n)$$

where $w_n = \dfrac{p(x_n)}{q(x_n)}$


In importance sampling, the proposal distribution is a probability distribution used to generate samples during the sampling process. It serves as an approximation to the target distribution from which you want to estimate properties or calculate expectations. The proposal distribution should ideally have a similar shape or be closely related to the target distribution to ensure accurate importance sampling estimates. The primary purpose of the proposal distribution is to efficiently explore the space of possible samples and assign appropriate weights to each sample. The importance weights, which are the ratios of the target distribution's density to the proposal distribution's density, are used to adjust the contribution of each sample to the final estimate.

In [5]:
def importance_sampling_markov_chain(transition_matrix, proposal_matrix, initial_state, num_samples, num_steps):
    samples = []
    weights = []
    
    for _ in range(num_samples):
        current_state = initial_state
        states = [current_state]
        weight = 1.0

        for _ in range(num_steps):
            transition_probabilities = transition_matrix[current_state]
            proposal_probabilities   = proposal_matrix[current_state]
            
            # Sample the next state using the proposal distribution
            next_state = np.random.choice(len(proposal_probabilities), p = proposal_probabilities)
            
            # Calculate the weight using the importance sampling ratio
            weight *= transition_probabilities[next_state] / proposal_probabilities[next_state]
            
            current_state = next_state
            states.append(current_state)

            # Stop sampling if a terminal state (e.g., Death) is reached
            if current_state == len(transition_matrix) - 1:
                break
        
        samples.append(states)
        weights.append(weight)
    
    # Normalize the weights
    weights /= np.sum(weights)
    
    return samples, weights

# Target distribution
# Transition matrix representing the Markov Chain
transition_matrix = np.array([
    [0.5, 0.4, 0.1, 0.0, 0.0],
    [0.0, 0.6, 0.3, 0.1, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.8, 0.2],
    [0.0, 0.0, 0.0, 0.0, 1.0]
])

# Proposal matrix representing the proposal distribution (can be different from the transition matrix)
# The proposal distribution is a probability distribution used to generate samples during the sampling 
# process. It serves as an approximation to the target distribution from which you want to estimate 
# properties or calculate expectations. The proposal distribution should ideally have a similar shape 
# or be closely related to the target distribution to ensure accurate importance sampling estimates.
proposal_matrix = np.array([
    [0.3, 0.3, 0.2, 0.1, 0.1],
    [0.1, 0.5, 0.2, 0.1, 0.1],
    [0.0, 0.1, 0.6, 0.2, 0.1],
    [0.0, 0.0, 0.1, 0.6, 0.3],
    [0.0, 0.0, 0.0, 0.0, 1.0]
])

initial_state = 0  # Starting from Stage 3
num_samples = 1000 # Number of importance samples
num_steps   = 10   # Number of steps to sample

# Perform importance sampling with the Markov Chain
sampled_states, importance_weights = importance_sampling_markov_chain(
    transition_matrix, proposal_matrix, initial_state, num_samples, num_steps)

In [6]:
for idx, state in enumerate(sampled_states):
    print(f'Patient {idx} with CKD Progression: \t {set(state)}')

Patient 0 with CKD Progression: 	 {0, 1, 2}
Patient 1 with CKD Progression: 	 {0, 2, 4}
Patient 2 with CKD Progression: 	 {0, 4}
Patient 3 with CKD Progression: 	 {0, 1, 3, 4}
Patient 4 with CKD Progression: 	 {0, 4}
Patient 5 with CKD Progression: 	 {0, 2, 3, 4}
Patient 6 with CKD Progression: 	 {0, 1, 2, 4}
Patient 7 with CKD Progression: 	 {0, 1, 2, 3, 4}
Patient 8 with CKD Progression: 	 {0, 2, 4}
Patient 9 with CKD Progression: 	 {0, 1, 2, 4}
Patient 10 with CKD Progression: 	 {0, 2, 3}
Patient 11 with CKD Progression: 	 {0, 4}
Patient 12 with CKD Progression: 	 {0, 1, 4}
Patient 13 with CKD Progression: 	 {0, 1, 2, 3, 4}
Patient 14 with CKD Progression: 	 {0, 1}
Patient 15 with CKD Progression: 	 {0, 2, 4}
Patient 16 with CKD Progression: 	 {0, 4}
Patient 17 with CKD Progression: 	 {0, 1, 3, 4}
Patient 18 with CKD Progression: 	 {0, 1, 3, 4}
Patient 19 with CKD Progression: 	 {0, 1, 3, 4}
Patient 20 with CKD Progression: 	 {0, 1, 3, 4}
Patient 21 with CKD Progression: 	 {0, 1, 2}

## Metropolis-Hastings Algorithm

In statistics and statistical physics, the Metropolis–Hastings algorithm is a Markov chain Monte Carlo (MCMC) method for obtaining a sequence of random samples from a probability distribution from which direct sampling is difficult. 

**Objective:** In the Metropolis-Hastings algorithm, the goal is to generate a Markov Chain that samples from a target distribution, which may be difficult to sample directly. The algorithm achieves this by using a proposal distribution to generate candidate states and then accepting or rejecting these candidates based on their acceptance probability.

Metropolis-Hastings algorithm requires the calculation of an acceptance probability. The acceptance probability is used to determine whether to accept or reject a proposed state transition during the sampling process. The acceptance probability is calculated as the ratio of the target distribution's density evaluated at the proposed state to the target distribution's density evaluated at the current state. The ratio is multiplied by a term that accounts for the proposal distribution's symmetry. The acceptance probability is defined as:

$$ \text{Acceptance Probability} = \min(1, \frac{{\text{{target\_density\_proposed}}}}{{\text{{target\_density\_current}}}} \cdot \frac{{\text{{proposal\_density\_current}}}}{{\text{{proposal\_density\_proposed}}}})$$


The acceptance probability ensures that the Markov Chain converges to the desired target distribution by accepting candidate states that improve the match with the target distribution and occasionally accepting states that decrease the match. This trade-off allows the algorithm to explore the state space and sample from the target distribution effectively. The acceptance probability acts as a balancing factor, ensuring that the Markov Chain explores regions of the state space with higher probabilities while allowing for occasional exploration of lower probability regions. It helps to avoid getting stuck in local optima and promotes convergence to the desired target distribution.

The acceptance or rejection of a proposed state transition is determined by comparing an acceptance ratio (or acceptance probability) to a randomly generated value. The accept-reject criterion can be summarized as follows:
1. Generate a candidate state transition from the current state using a proposal distribution.
2. Calculate the acceptance ratio, which is the ratio of the target distribution's density evaluated at the proposed state to the target distribution's density evaluated at the current state. This ratio is multiplied by a term that accounts for the proposal distribution's symmetry.
3. Generate a random value, typically from a uniform distribution between 0 and 1.
4. **If the acceptance ratio is greater than or equal to the random value (typical: uniform distribution), accept the proposed state transition. Otherwise, reject it and retain the current state.**


The code below implements the Metropolis-Hastings algorithm for sampling from the Markov Chain. Starting from the initial state, the algorithm proposes new states based on the transition probabilities and decides whether to accept or reject the proposed state based on an acceptance probability. The process is repeated for the desired number of samples and steps.



In [7]:
import numpy as np

def metropolis_hastings_markov_chain(transition_matrix, initial_state, num_samples, num_steps):
    samples = []
    
    current_state = initial_state
    for _ in range(num_samples):
        states = [current_state]

        for _ in range(num_steps):
            transition_probabilities = transition_matrix[current_state]
            
            # Propose a new state based on the transition probabilities
            next_state = np.random.choice(len(transition_probabilities), p=transition_probabilities)
            
            # Calculate the acceptance probability
            # weight from the importance sampling
            acceptance_prob = min(1, transition_probabilities[next_state] / transition_probabilities[current_state])
            
            # Use criteria 
            # Accept or reject the proposed state based on the acceptance probability
            # Uniform or Poisson
            if np.random.uniform() < acceptance_prob:
                current_state = next_state
            
            states.append(current_state)

            # Stop sampling if a terminal state (e.g., Death) is reached
            if current_state == len(transition_matrix) - 1:
                break
        
        samples.append(states)
    
    return samples

# Transition matrix representing the Markov Chain
transition_matrix = np.array([
    [0.5, 0.4, 0.1, 0.0, 0.0],
    [0.0, 0.6, 0.3, 0.1, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.8, 0.2],
    [0.0, 0.0, 0.0, 0.0, 1.0]
])

initial_state = 0  # Starting from Stage 3
num_samples   = 10 # Number of Metropolis-Hastings samples
num_steps     = 10  # Number of steps to sample # Number of years

# Perform Metropolis-Hastings sampling with the Markov Chain
sampled_states = metropolis_hastings_markov_chain(transition_matrix, initial_state, num_samples, num_steps)

for idx, state in enumerate(sampled_states):
    print(f'Patient {idx} with CKD Progression: \t {state}')

Patient 0 with CKD Progression: 	 [0, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3]
Patient 1 with CKD Progression: 	 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Patient 2 with CKD Progression: 	 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Patient 3 with CKD Progression: 	 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Patient 4 with CKD Progression: 	 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Patient 5 with CKD Progression: 	 [3, 3, 3, 4]
Patient 6 with CKD Progression: 	 [4, 4]
Patient 7 with CKD Progression: 	 [4, 4]
Patient 8 with CKD Progression: 	 [4, 4]
Patient 9 with CKD Progression: 	 [4, 4]


By comparing the acceptance ratio to a random value, the Metropolis-Hastings algorithm balances exploration and exploitation. It allows for occasional acceptance of state transitions that decrease the match with the target distribution, which helps the algorithm explore the state space and avoid getting stuck in local optima. The accept-reject criterion plays a crucial role in the algorithm's convergence to the desired target distribution and ensures that the Markov Chain generated by the algorithm approximates the target distribution effectively.

In [9]:
import numpy as np

def importance_sampling_markov_chain(transition_matrix, pdf_func, initial_state, num_samples, num_steps):
    samples = []
    weights = []
    
    for _ in range(num_samples):
        current_state = initial_state
        states = [current_state]
        weight = 1.0

        for _ in range(num_steps):
            transition_probabilities = transition_matrix[current_state]
            
            # Generate a sample from the proposal distribution using the given PDF
            proposal_probabilities = pdf_func(current_state)
            next_state = np.random.choice(len(proposal_probabilities), p=proposal_probabilities)
            
            # Calculate the weight using the importance sampling ratio
            weight *= transition_probabilities[next_state] / proposal_probabilities[next_state]
            
            current_state = next_state
            states.append(current_state)

            # Stop sampling if a terminal state (e.g., Death) is reached
            if current_state == len(transition_matrix) - 1:
                break
        
        samples.append(states)
        weights.append(weight)
    
    # Normalize the weights
    weights /= np.sum(weights)
    
    return samples, weights

# Transition matrix representing the Markov Chain
transition_matrix = np.array([
    [0.5, 0.4, 0.1, 0.0, 0.0],
    [0.0, 0.6, 0.3, 0.1, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.8, 0.2],
    [0.0, 0.0, 0.0, 0.0, 1.0]
])

initial_state = 0  # Starting from Stage 3
num_samples = 1000  # Number of importance samples
num_steps = 10  # Number of steps to sample

# Define the proposal probability density function (PDF)
def proposal_pdf(state):
    # Example: Uniform distribution as the proposal PDF
    num_states = len(transition_matrix)
    probabilities = np.full(num_states, 1/num_states)
    return probabilities

# Perform importance sampling with the Markov Chain
sampled_states, importance_weights = importance_sampling_markov_chain(
    transition_matrix, proposal_pdf, initial_state, num_samples, num_steps
)

# Print the sampled states and their corresponding importance weights for the first sample
print("Sampled States (First Sample):", sampled_states[0])
print("Importance Weights (First Sample):", importance_weights[0])


Sampled States (First Sample): [0, 3, 4]
Importance Weights (First Sample): 0.0


In [10]:
proposal_pdf(state)

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [13]:
for idx, state in enumerate(sampled_states):
    print(f'Patient {idx} with CKD Progression: \t {set(state)}')

Patient 0 with CKD Progression: 	 {0, 3, 4}
Patient 1 with CKD Progression: 	 {0, 1, 2, 4}
Patient 2 with CKD Progression: 	 {0, 1, 4}
Patient 3 with CKD Progression: 	 {0, 2, 4}
Patient 4 with CKD Progression: 	 {0, 1, 4}
Patient 5 with CKD Progression: 	 {0, 2, 3, 4}
Patient 6 with CKD Progression: 	 {0, 2, 4}
Patient 7 with CKD Progression: 	 {0, 3, 4}
Patient 8 with CKD Progression: 	 {0, 1, 2, 3, 4}
Patient 9 with CKD Progression: 	 {0, 4}
Patient 10 with CKD Progression: 	 {0, 1, 2, 4}
Patient 11 with CKD Progression: 	 {0, 1, 3, 4}
Patient 12 with CKD Progression: 	 {0, 4}
Patient 13 with CKD Progression: 	 {0, 1, 2, 4}
Patient 14 with CKD Progression: 	 {0, 1, 2, 3, 4}
Patient 15 with CKD Progression: 	 {0, 2, 4}
Patient 16 with CKD Progression: 	 {0, 2, 3, 4}
Patient 17 with CKD Progression: 	 {0, 4}
Patient 18 with CKD Progression: 	 {0, 4}
Patient 19 with CKD Progression: 	 {0, 3, 4}
Patient 20 with CKD Progression: 	 {0, 2, 3, 4}
Patient 21 with CKD Progression: 	 {0, 2, 4}

### General Implementation: Metropolis-Hastings Algorithm

$$ f(x) = \frac{e^{\frac{-(x-5)^2}{8}}}{8\pi} $$

In [31]:
import numpy as np

def target_distribution(x):
    """Target distribution: Gaussian with mean 5 and standard deviation 2."""
    return np.exp(-(x - 5) ** 2 / 8) / np.sqrt(8 * np.pi)

def proposal_distribution():
    """Proposal distribution: Uniform distribution between 0 and 10."""
    return np.random.uniform(0, 10)

def importance_sampling(n_samples):
    """Implementation of Importance Sampling."""
    samples = np.zeros(n_samples)
    weights = np.zeros(n_samples)

    for i in range(n_samples):
        sample = proposal_distribution()
        weight = target_distribution(sample) / proposal_distribution()
        samples[i] = sample
        weights[i] = weight

    weights /= np.sum(weights)  # Normalize the weights
    estimated_mean = np.sum(samples * weights)

    return estimated_mean

def metropolis_hastings(n_samples, burn_in=1000):
    """Implementation of Metropolis-Hastings."""
    samples = np.zeros(n_samples + burn_in)
    accepted = 0
    current_sample = proposal_distribution()

    for i in range(n_samples + burn_in):
        proposal = proposal_distribution()
        acceptance_prob = min(1, target_distribution(proposal) / target_distribution(current_sample))

        if np.random.rand() < acceptance_prob:
            current_sample = proposal
            accepted += 1

        samples[i] = current_sample

    samples = samples[burn_in:]
    estimated_mean = np.mean(samples)

    return estimated_mean, accepted / (n_samples + burn_in)

# Testing the implementations
np.random.seed(42)
n_samples = 10000

# Importance Sampling
estimated_mean_importance = importance_sampling(n_samples)
print("Importance Sampling:")
print("Estimated Mean:", estimated_mean_importance)

# Metropolis-Hastings
estimated_mean_mh, acceptance_ratio = metropolis_hastings(n_samples)
print("\nMetropolis-Hastings:")
print("Estimated Mean:", estimated_mean_mh)
print("Acceptance Ratio:", acceptance_ratio)

Importance Sampling:
Estimated Mean: 4.815660868077247

Metropolis-Hastings:
Estimated Mean: 4.984573440505409
Acceptance Ratio: 0.615
